In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy import geocoders
import folium
from folium import IFrame

In [15]:
colleges_data = pd.read_excel('Discrete Colleges.xlsx')
colleges_data.shape

(287, 2)

In [16]:
colleges_data.drop(['S No.'], axis = 1, inplace = True)
colleges_data.head()

,College/Organization
0,Amritsar College of Engineering & Technology (...
1,"Amal Jyothi College of Engineering (AJCE), Kerala"
2,"Ambedkar University, Delhi"
3,"Amity University (AU), Jaipur"
4,"Amrita School Of Engineering (ASE), Bangalore"


In [17]:
colleges_data['Location'] = colleges_data['College/Organization'].apply(lambda x : x.split(',')[-1].strip())
colleges_data.head()

,College/Organization,Location
0,Amritsar College of Engineering & Technology (...,Punjab
1,"Amal Jyothi College of Engineering (AJCE), Kerala",Kerala
2,"Ambedkar University, Delhi",Delhi
3,"Amity University (AU), Jaipur",Jaipur
4,"Amrita School Of Engineering (ASE), Bangalore",Bangalore


In [18]:
unique_loc, count = np.unique(colleges_data['Location'], return_counts=True)
len(unique_loc)

110

In [19]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises", timeout=3)
loc_list = []
lat = []
long = []
for loc in unique_loc:
    location = geolocator.geocode(loc)
    loc_list.append(loc)
    lat.append(location.latitude)
    long.append(location.longitude)
    print(loc, location.latitude, location.longitude)

Ahmedabad 23.0216238 72.5797068
Aligarh 27.87698975 78.13729027600994
Amritsar 31.6343083 74.8736788
Andhra Pradesh 15.9240905 80.1863809
Aurangabad 19.877263 75.3390241
Bangalore 12.9791198 77.5912997
Banglore 18.0056901 79.5715088
Bathinda 30.17911535 75.04710156312628
Bhubaneswar 20.2667774 85.8435592
Bihar 25.6440845 85.906508
Bilaspur 28.8680526 79.29838501460121
Bombay 18.9387711 72.8353355
Calcutta 22.5677459 88.3476023
Chandigarh 30.7194022 76.7646552
Chattisgarh 21.6637359 81.8406351
Chennai 13.0801721 80.2838331
Coimbatore 11.0018115 76.9628425
Dehradun 30.3255646 78.0436813
Delhi 28.6517178 77.2219388
Gandhinagar 23.2232877 72.6492267
Ghaziabad 28.711241 77.4445372
Goa 15.3004543 74.0855134
Gorakhpur 26.67132865 83.36458284327038
Government Of Jharkhand 23.4875593 86.3100914
Government college of engineering kannur 11.98543625 75.38148295333137
Greater Noida 28.4670734 77.5137649
Gujarat 22.41540825 72.03149703699282
Gurgaon 28.42826235 77.00270014657752
Guwahati 26.1805978 

In [23]:
loc_df = pd.DataFrame(columns=['Location', 'Latitude', 'Longitude'])
loc_df['Location'] = loc_list
loc_df['Latitude'] = lat
loc_df['Longitude'] = long
loc_df.to_csv('College Locations.csv', index = False)
loc_df.head()

,Location,Latitude,Longitude
0,Ahmedabad,23.021624,72.579707
1,Aligarh,27.876990,78.137290
2,Amritsar,31.634308,74.873679
3,Andhra Pradesh,15.924091,80.186381
4,Aurangabad,19.877263,75.339024


In [50]:
df_merged = pd.merge(colleges_data, loc_df, on = 'Location')
df_merged.shape

(287, 4)

In [51]:
df_merged.head()

,College/Organization,Location,Latitude,Longitude
0,Amritsar College of Engineering & Technology (...,Punjab,30.929321,75.500484
1,Amritsar College of Engineering & Technology (...,Punjab,30.929321,75.500484
2,Panjab University Swami Sarvanand Giri (PU SSG...,Punjab,30.929321,75.500484
3,Thapar Institute of Engineering and Technology...,Punjab,30.929321,75.500484
4,Thapar Institute of Engineering and Technology...,Punjab,30.929321,75.500484


In [81]:
map1 = folium.Map(
    location=[28.6517178, 77.2219388],   # initialize with Delhi Co-ordinates 28.6517178 77.2219388
    tiles='cartodbpositron',
    zoom_start=5
)

title_html = '''
             <h2 align="center" style="font-size:24px"><b>TECHADROIT4.0 Participation</b></h2>
             <h4 align="center" style="font-size:20px"><b>900+ Registrations</b></h4>
             '''
map1.get_root().html.add_child(folium.Element(title_html))

df = df_merged.copy()
df.apply(lambda row:folium.Marker(
    location=[row["Latitude"], row["Longitude"]],
    popup = ('<h5>'+ 'City: ' + str(row['Location'] + '<br><br>' + 'College: ' + row['College/Organization']) + '</h5>'),
    ).add_to(map1), axis=1)
map1

In [82]:
map1.save('TECHADROIT4.0 Participation.html')

In [83]:
df_merged.to_csv('Colleges_location_combined.csv', index = False)